# What Can Individuals Do to Prevent the Spread of COVID?

# Table of Contents
1. [Import Statement](#import)
2. [Original Data](#original)
3. [Merging Data](#merge)
4. [Data Manipulation](#manipulation)
5. [Data Exploration](#exploration)
6. [Visualizing Cases Per Population with Different Mask Usage Levels](#visual) 
7. [Linear Regression](#linear)

** To run this jupyter notebook, please install the modules that are listed below. (All of the modules used were covered in class) In addition, please change the file directory for the mask.csv, us_county.csv, and population.csv to the location in which your files are stored. 

### Import Statements <a class="anchor" id="import"></a>

In [24]:
# Import Statements for Modules
# All the modules we used were covered in this class
import pandas as pd 
import numpy as np
import plotly.express as px
import statsmodels.formula.api as smf 
import statsmodels.api as sm 

In [1]:
# Please change the directory of files accordingly for mask, us_county, and population data 
mask = pd.read_csv("C:/Users/jongm/Desktop/mask_usage.csv")
us_county = pd.read_csv("C:/Users/jongm/Desktop/us-counties.csv")
population = pd.read_csv("C:/Users/jongm/Desktop/population.csv")

### Original Data <a class="anchor" id="original"></a>

In [15]:
# Viewing the population table
population.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [16]:
# Viewing the mask table
mask.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [17]:
# Viewing the us_county table (this shows the counties with number of cases and deaths)
us_county.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


### Merging Data <a class="anchor" id="merge"></a>

In [18]:
# Merging the US_County to the Mask Usage Data on FIPS and COUNTYFP
merged = us_county.merge(mask, left_on='fips', right_on='COUNTYFP')

# Merging the Merged data with the population data on COUNTYFP and CountyId
merged = merged.merge(population, left_on='COUNTYFP', right_on='CountyId')
merged.head()

,date,county,state,fips,cases,deaths,COUNTYFP,NEVER,RARELY,SOMETIMES,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,2020-01-21,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.7,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4
1,2020-01-22,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.7,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4
2,2020-01-23,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.7,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4
3,2020-01-24,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.7,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4
4,2020-01-25,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.7,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4


### Data Manipulation <a class="anchor" id="manipulation"></a>

In [19]:
# Calculating Cases per Population 
merged['caseperpop'] = merged['cases']/merged['TotalPop']
merged.head()

,date,county,state,fips,cases,deaths,COUNTYFP,NEVER,RARELY,SOMETIMES,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,caseperpop
0,2020-01-21,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001
1,2020-01-22,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001
2,2020-01-23,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001
3,2020-01-24,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001
4,2020-01-25,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,1.6,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001


In [20]:
# Setting 3 boundaries depending on how frequently masks are worn in each county
usage = []

'''If the percentage of people who wear masks all the time are below 33%, the county is considered to be LOW, 
if the percentage of people who wear masks are below 66%, the county is considered MEDIUM, and the remaining 
counties are considered as HIGH''' 
for row in merged['ALWAYS']:
    if row < .33:
        usage.append('LOW')
    elif row < .66:
        usage.append('MEDIUM')
    elif row < 1:
        usage.append('HIGH')

# Adding the calculated categories to a new column called mask_usage
merged['mask_usage'] = usage
merged.head()

,date,county,state,fips,cases,deaths,COUNTYFP,NEVER,RARELY,SOMETIMES,...,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,caseperpop,mask_usage
0,2020-01-21,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001,HIGH
1,2020-01-22,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001,HIGH
2,2020-01-23,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001,HIGH
3,2020-01-24,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001,HIGH
4,2020-01-25,Snohomish,Washington,53061.0,1,0,53061,0.017,0.014,0.056,...,5.2,31.8,390186,81.9,12.5,5.4,0.2,5.4,0.000001,HIGH


In [22]:
# Grouping by mask_usage and dates on Cases per population 
# This demonstrates how different counties with varying levels of mask usage affect cases per population
mask_group = merged.groupby(['mask_usage','date'])['caseperpop'].mean()
mask_group = mask_group.reset_index()
mask_group.head()

,mask_usage,date,caseperpop
0,HIGH,2020-01-21,1.295498e-06
1,HIGH,2020-01-22,1.295498e-06
2,HIGH,2020-01-23,1.295498e-06
3,HIGH,2020-01-24,7.431954e-07
4,HIGH,2020-01-25,6.010887e-07


### Data Exploration <a class="anchor" id="exploration"></a>

In [26]:
# Exploring how cases in Dekalb County have changed over time
dekalb = merged[merged['county'] == 'DeKalb']
dekalb = dekalb[dekalb['state'] == 'Georgia']
dekalb.head()

,date,county,state,fips,cases,deaths,COUNTYFP,NEVER,RARELY,SOMETIMES,...,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,caseperpop,mask_usage
29482,2020-03-09,DeKalb,Georgia,13089.0,2,0,13089,0.036,0.028,0.075,...,6.2,31.9,365630,81.6,13.2,5.1,0.2,8.5,0.000003,HIGH
29483,2020-03-10,DeKalb,Georgia,13089.0,2,0,13089,0.036,0.028,0.075,...,6.2,31.9,365630,81.6,13.2,5.1,0.2,8.5,0.000003,HIGH
29484,2020-03-11,DeKalb,Georgia,13089.0,4,0,13089,0.036,0.028,0.075,...,6.2,31.9,365630,81.6,13.2,5.1,0.2,8.5,0.000005,HIGH
29485,2020-03-12,DeKalb,Georgia,13089.0,4,0,13089,0.036,0.028,0.075,...,6.2,31.9,365630,81.6,13.2,5.1,0.2,8.5,0.000005,HIGH
29486,2020-03-13,DeKalb,Georgia,13089.0,4,0,13089,0.036,0.028,0.075,...,6.2,31.9,365630,81.6,13.2,5.1,0.2,8.5,0.000005,HIGH


In [35]:
# Plotting Cases in Dekalb over Time
fig1 = px.scatter(x=dekalb['date'],y=dekalb['cases'], title = "Cases Over Time in Dekalb County", 
                  labels={"x":"Time", "y":"Cases"})
fig1.show()

In [38]:
# Creating a function to look at how cases vary for a county that the user is interested in 
def county_case(county, state):
    try:
        interest = merged[merged['county'] == county]
        interest = interest[interest['state'] == state]
        fig1 = px.scatter(x=interest['date'],y=interest['cases'], title = "Cases Over Time in " + county + " County", 
                  labels={"x":"Time", "y":"Cases"})
        return fig1.show()
    except:
        print("Could not find the county and state. Please enter a county and a state that is in the United States")

In [39]:
# User can input the county and then the state in quotations like the example below to find the state and county they are interested in 
county_case('Cobb','Georgia')

### Visualizing Cases Per Population with Different Mask Usage Levels <a class="anchor" id="visual"></a>

In [40]:
# Creating a scatter plot that looks across time for cases per population (colored by mask_usage)
fig1 = px.scatter(x=mask_group['date'],y=mask_group['caseperpop'], color = mask_group['mask_usage'],
                  title = "How Mask Usage Affects COVID Cases", labels={"x":"Time", "y":"Cases per Population", "color":"Mask Usage"})
fig1.show()

### Linear Regression on Cases Per Population <a class="anchor" id="linear"></a>

In [25]:
# Creating a simple linear regression on casesperpopulation and mask_usage (represented by the column ALWAYS)
model = smf.ols('caseperpop ~ ALWAYS', data = merged)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             caseperpop   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     111.3
Date:                Sun, 15 Nov 2020   Prob (F-statistic):           5.12e-26
Time:                        18:09:27   Log-Likelihood:             1.9912e+06
No. Observations:              703013   AIC:                        -3.982e+06
Df Residuals:                  703011   BIC:                        -3.982e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0115   6.02e-05    191.555      0.0

In [8]:
# Exploring how other factors like income, working from home, transit, and mask usage will affect cases per population
model = smf.ols('caseperpop ~ IncomePerCap+ WorkAtHome+ Transit + ALWAYS', data = merged)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             caseperpop   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4585.
Date:                Sun, 15 Nov 2020   Prob (F-statistic):               0.00
Time:                        17:41:59   Log-Likelihood:             2.0002e+06
No. Observations:              703013   AIC:                        -4.000e+06
Df Residuals:                  703008   BIC:                        -4.000e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0206   9.06e-05    227.569   